In [18]:
import gradio as gr
from dotenv import load_dotenv
import urllib
import google.generativeai as genai
# 특정 문자열 추출을 위해 import
import re

load_dotenv() # load "/.env" file.
genai.configure() # load GOOGLE_API_KEY from "/.env" file

In [19]:
# func1 - 조리 방법 알아내기
## 사진을 넣으면, 출력으로 1.음식이름 2.필요한 재료 3.필요한 조리도구 4.조리 순서 5.kcal

#  사진 분석 결과 출력을 위해 입력할 지문
def description() :
    txt = f"사진 속 음식의 이름이 무엇이며 사진 속 음식을 만들기 위해 필요한 재료, 필요한 조리도구, 조리 순서 그리고 재료를 통해 사진 속 음식의 칼로리(kcal)를 예상하여 '음식이름', '재료', '조리도구', '조리순서', '칼로리' 총 5가지 항목으로 나누어 알려주세요."
    return txt

# 문장 추출을 위한 기준 인덱스 선정
def find_index(text, index) :
    text = text[index+2:] # \n 두개 스킵
    to_index = text.find("\n\n")
    return text, to_index

# # 5개 출력
# 음식 이름 추출
def sep_foodName(text, index) :
    name = text[:index].replace(":","").split() # 1.문단 끝부분까지 string 가져오고, 2.':'있으면 제거하고 공백기준으로 리스트 형태로 분리
    name = ' '.join(name[1:]) # 리스트->문자열로 변경하기 위해 join(e.g. [비빔, 밥]) / name[0]: 음식이름
    return name

# 재료 추출
def sep_grad(text, index) :
    # index 및 원하는 2번째 문장 찾기
    text, to_index = find_index(text, index)
    grad_part = text[:to_index] # 문단 끝부분까지 지문 가져오기
    grad = grad_part[grad_part.find("\n")+1:]
    return grad

# 조리도구 추출
def sep_equip(text, index) :
    # index 및 원하는 3번째 문장 찾기
    for i in range(2):
        text, index = find_index(text, index)
    equip_part = text[:index]
    equip = equip_part[equip_part.find("\n")+1:]
    return equip
    
# 조리순서 추출
def sep_recipe(text, index) :
    # index 및 원하는 4번째 문장 찾기
    for i in range(3):
        text, index = find_index(text, index)
    recipe_part = text[:index]
    recipe = recipe_part[recipe_part.find("\n")+1:]
    return recipe

# 칼로리 추출
def sep_kcal(text, index) :
    # index 및 원하는 5번째 문장 찾기
    for i in range(4):
        text, index = find_index(text, index)
    kcal = text[:index].replace(":","").split()
    kcal = ' '.join(kcal[1:])
    return kcal
    
# 프로세스
def process(image) :
    # gemnai 모델 불러오기
    vmodel = genai.GenerativeModel('gemini-pro-vision')
    # 프롬프트
    prompt = description()
    response = vmodel.generate_content([prompt, image])
    
    # 초기 문장 범위 설정
    text = response.text.replace("**", "") # 전체 특수 문자를 공백 대체
    index = text.find("\n\n") # 문단 끝 부분 찾기
    
    return sep_foodName(response.text, index), sep_grad(response.text, index), sep_equip(response.text, index), sep_recipe(response.text, index), sep_kcal(response.text, index)

demo = gr.Interface(
    fn=process, 
    inputs=[gr.Image(type="pil")], 
    outputs=["text", "text", "text", "text", "text"]
	)

if __name__ == "__main__":
    demo.launch()
    

Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.
